In [ ]:

%pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu128


In [ ]:
import torch
print(torch.__version__)  
print(torch.cuda.is_available())  
print(torch.cuda.current_device())  
print(torch.cuda.get_device_name(0)) 

In [ ]:
%pip install opencv-python-headless
%pip install opencv-python
%pip install scikit-image
%pip install scikit-learn
%pip install tqdm
%pip install sympy==1.13.3


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

In [ ]:
base_dir = "D:/Licenta/Datasets/Parkinson_s Drawings/augmented_combined/"
train_dir = os.path.join(base_dir, "training")
test_dir = os.path.join(base_dir, "testing")

batch_size = 32
epochs = 15
lr = 1e-4
img_size = 224

train_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

test_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])


In [ ]:
train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
test_dataset = datasets.ImageFolder(test_dir, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

print("Classes:", train_dataset.classes)

In [ ]:
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)


In [ ]:
import matplotlib.pyplot as plt

train_losses = []
train_accuracies = []

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total

    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc)

    print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_acc*100:.2f}%")


In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, marker='o')
plt.title("Train Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, marker='o', color='green')
plt.title("Train Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")

plt.tight_layout()
plt.show()

In [ ]:
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        y_pred.extend(preds)
        y_true.extend(labels.numpy())

from sklearn.metrics import classification_report
print("\n CLASSIFICATION REPORT:")
print(classification_report(y_true, y_pred, target_names=train_dataset.classes))


In [ ]:
torch.save(model.state_dict(), "resnet50_parkinson.pth")
print("Model salvat în resnet50_parkinson.pth")


In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np

class_names = train_dataset.classes

images_list = []
labels_list = []
preds_list = []

model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images.to(device))
        preds = torch.argmax(outputs, dim=1)

        images_list.extend(images)
        labels_list.extend(labels)
        preds_list.extend(preds.cpu())


num_samples = 100 

cols = 5
rows = num_samples // cols
plt.figure(figsize=(3 * cols, 3 * rows))

for i in range(num_samples):
    idx = random.randint(0, len(images_list) - 1)
    img = images_list[idx].permute(1, 2, 0).numpy()
    img = (img * 0.5 + 0.5).clip(0, 1)

    true_label = class_names[labels_list[idx]]
    pred_label = class_names[preds_list[idx]]

    plt.subplot(rows, cols, i+1)
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"T:{true_label}\nP:{pred_label}", fontsize=8,
              color='green' if true_label == pred_label else 'red')

plt.suptitle("Real vs Predicted (100 random)", fontsize=18)
plt.tight_layout()
plt.show()



In [ ]:
wrong_idx = [i for i in range(len(preds_list)) if preds_list[i] != labels_list[i]]

print(f"Total greșeli: {len(wrong_idx)} din {len(preds_list)}")

plt.figure(figsize=(15, 6))
for i, idx in enumerate(wrong_idx[:5]):  
    img = images_list[idx].permute(1, 2, 0).numpy()
    img = (img * 0.5 + 0.5).clip(0, 1)

    true_label = class_names[labels_list[idx]]
    pred_label = class_names[preds_list[idx]]

    plt.subplot(1, 5, i+1)
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"T: {true_label}\nP: {pred_label}", color='red')

plt.suptitle("Predicții greșite", fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(labels_list, preds_list)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)

plt.figure(figsize=(6, 6))
disp.plot(cmap=plt.cm.Blues, values_format='d')
plt.title("Confusion Matrix")
plt.show()

In [ ]:
from torchvision.transforms.functional import normalize, resize, to_tensor
from torchvision.transforms import ToPILImage
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torchcam.methods import GradCAM

cam_extractor = GradCAM(model, target_layer="layer4")

model.eval()

def show_gradcam(img_tensor, label_idx):
    img_tensor = img_tensor.unsqueeze(0).to(device)

    out = model(img_tensor)
    pred_class = out.argmax().item()
    activation_map = cam_extractor(pred_class, out)

    cam = activation_map[0].cpu().numpy()
    cam = np.uint8(255 * cam.squeeze())  
    cam = cv2.resize(cam, (img_tensor.shape[3], img_tensor.shape[2]))


    heatmap = cv2.applyColorMap(cam, cv2.COLORMAP_JET)
    img_np = img_tensor.squeeze().permute(1, 2, 0).detach().cpu().numpy()
    img_np = (img_np * 0.5 + 0.5).clip(0, 1)
    img_np = np.uint8(img_np * 255)
    overlay = cv2.addWeighted(img_np, 0.6, heatmap, 0.4, 0)

    plt.imshow(overlay)
    plt.axis("off")
    plt.title(f"True: {class_names[label_idx]} | Pred: {class_names[pred_class]}")
    plt.show()

In [ ]:
sample_idx = random.randint(0, len(images_list)-1)
sample_image = images_list[sample_idx]
sample_label = labels_list[sample_idx]

show_gradcam(sample_image, sample_label)